In [4]:
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import backend as K
from keras.datasets import cifar10
from keras.applications import resnet50
import numpy as np
from keras.models import load_model

from utils import display_image
from Generator import Generator
from ProjectedGradientDescent import ProjectedGradientDescent

In [19]:
def preprocess(image, norm=False, size=224):
    image = tf.cast(image, tf.float32)
    if norm:
        image = image /255
    image = tf.image.resize(image, (size, size))
    image = image[None, ...]
    return image

In [6]:
if False:
    pretrained_model = tf.keras.applications.ResNet50(include_top=True, weights='imagenet')
    pretrained_model.trainable = False
else:
    pretrained_model = load_model('cifar_model_test.h5')
    pretrained_model.trainable = False


c:\users\lgran\appdata\local\programs\python\python36\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [16]:
image_path = tf.keras.utils.get_file('YellowLabradorLooking_new.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')
image_raw = tf.io.read_file(image_path)
image = tf.image.decode_image(image_raw)

image = preprocess(image)
print(image.shape)

(1, 224, 224, 3)


In [28]:
pretrained_model.get_config()

{'name': 'model_1',
 'layers': [{'name': 'input_0',
   'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 3, 32, 32),
    'dtype': 'float32',
    'sparse': False,
    'name': 'input_0'},
   'inbound_nodes': []},
  {'name': '321_pad',
   'class_name': 'ZeroPadding2D',
   'config': {'name': '321_pad',
    'trainable': True,
    'dtype': 'float32',
    'padding': ((1, 1), (1, 1)),
    'data_format': 'channels_first'},
   'inbound_nodes': [[['input_0', 0, 0, {}]]]},
  {'name': '321',
   'class_name': 'Conv2D',
   'config': {'name': '321',
    'trainable': True,
    'dtype': 'float32',
    'filters': 64,
    'kernel_size': (3, 3),
    'strides': (1, 1),
    'padding': 'valid',
    'data_format': 'channels_first',
    'dilation_rate': (1, 1),
    'activation': 'linear',
    'use_bias': False,
    'kernel_initializer': {'class_name': 'Zeros', 'config': {}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer':

In [32]:
seed_image = tf.reshape(seed_image, [1, 3, 32, 32])
preds = pretrained_model.predict(seed_image, steps=1)

In [33]:
# print('Predicted:', resnet50.decode_predictions(preds, top=3)[0])
preds

array([[ 23.691584 , -19.288782 ,  -2.6492743,   5.052263 ,  19.403622 ,
        -21.785276 ,   7.386835 ,   3.710042 ,   6.488927 , -22.005743 ]],
      dtype=float32)

In [ ]:
target_class = 3
n_classes = 1000
prob = tf.one_hot([target_class], n_classes)

pgd = ProjectedGradientDescent(pretrained_model, image, prob)
adv_image = pgd.gen_adv_example(showImage=True)

preds = pretrained_model.predict(adv_image)
print('Predicted:', resnet50.decode_predictions(preds, top=3)[0])

In [41]:
generator = Generator()
generator.calc_class_stats()

seed = generator.generate_seed(5)
seed = seed.reshape(1, 3, 32, 32)
# seed_image = preprocess(seed, size=32)
seed_image = tf.reshape(seed_image, [1, 3, 32, 32])
# display_image(seed, "Seeded image")
# display_image(seed_image, "Upsampled seed with tfs bilinear interpolation")

In [42]:
print(seed_image.shape)

(1, 3, 32, 32)


In [43]:
# preds = pretrained_model.predict(seed_image)
# print('Predicted:', resnet50.decode_predictions(preds, top=3)[0])

target_class = 5
n_classes = 10
prob = tf.one_hot([target_class], n_classes)
pgd = ProjectedGradientDescent(pretrained_model, seed_image, prob)
adv_seed = pgd.gen_adv_example(showImage=False)

# preds = pretrained_model.predict(adv_seed)
# print('Predicted:', resnet50.decode_predictions(preds, top=3)[0])

AttributeError: 'Tensor' object has no attribute 'numpy'

In [ ]:
generator.visualize_example_seeds()